 **** EfficientNetB0 - without data preprocessing --> only resize, rescale and data aug

In [ ]:
#new
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import random
import cv2 
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential, load_model
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications 

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
train_dir = '../input/aptos2019-blindness-detection/train_images'
#test_dir = '../input/aptos2019-blindness-detection/test_images'
train_csv=pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
#test_csv=pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

In [ ]:
print(len(train_csv))
#print(len(test_csv))

In [ ]:
train_csv['diagnosis'].value_counts()
train_csv['diagnosis'].hist()

In [ ]:
train=[]
trainlabel=[]
for im in tqdm(os.listdir(train_dir)):
    image=load_img(os.path.join(train_dir,im), grayscale=False, color_mode='rgb', target_size=(224,224))
    image=img_to_array(image)
    image=image/255.0
    train+=[image]
    trainlabel+=[train_csv[train_csv['id_code']==im[0:-4]]['diagnosis'].iat[0]]

In [ ]:
print(len(train))
print(len(trainlabel))

In [ ]:
train=np.array(train)
trainlabel=np.array(trainlabel)

In [ ]:
print(train.shape)
print(trainlabel.shape)

In [ ]:
trainlabel2=to_categorical(trainlabel)

In [ ]:
import pickle

pickle_out = open("train.pickle","wb")
pickle.dump(train, pickle)
pickle_out.close()

pickle_out = open("trainlabel2.pickle","wb")
pickle.dump(trainlabel2, pickle_out)
pickle_out.close()

In [ ]:
##how to open this pickle file
pickle_in = open("train.pickle", "rb")
train = pickle.load(pickle_in)

In [ ]:
trainx,testx,trainy,testy=train_test_split(train,trainlabel2,test_size=0.2,random_state=100)

In [ ]:
print(trainx.shape)
print(trainy.shape)
print(testx.shape)
print(testy.shape)

In [ ]:
datagen = ImageDataGenerator(zoom_range=0.1,  
        rotation_range = 30,
        horizontal_flip=True, 
        vertical_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,                     
    )

In [ ]:
eff = applications.EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224,224,3))
densen = applications.DenseNet201(include_top=False, weights='imagenet', input_shape=(224,224,3))

In [ ]:
from keras.applications import VGG16
from keras.optimizers import Adam
from keras import layers


def build_model1():
    model = Sequential()
    model.add(eff)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(256))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(256, activation = 'softmax'))
    model.add(layers.Dense(1))
    
    model.compile(
        loss='mse',
        optimizer=Adam(lr=0.0001),
        metrics=['accuracy']
    )
    
    return model
              
              

def build_model2():
    model = Sequential()
    model.add(densen)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(256))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(256, activation = 'softmax'))
    model.add(layers.Dense(1))
    
    model.compile(
        loss='mse',
        optimizer=Adam(lr=0.0001),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
model1 = build_model1()
model1.summary()

In [ ]:
model2 = build_model2()
model2.summary()

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from PIL import Image

import tensorflow as tf
from keras import layers
from tensorflow.keras import applications 
from keras.applications import MobileNet
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras import models


from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix

from tqdm import tqdm


#history = model.fit(
 #   data_generator,
  #  validation_data=(testx,testy),
   # epochs=5)

his = model1.fit(
    datagen.flow(trainx,trainy,batch_size=32),
    validation_data=(testx,testy),
    epochs=20,
    steps_per_epoch=trainx.shape[0] / 32 
    )

In [ ]:
his1= model2.fit(
    datagen.flow(trainx,trainy,batch_size=32),
    validation_data=(testx,testy),
    epochs=20,
    steps_per_epoch=trainx.shape[0] / 32
    )

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
train = []
trainlabel = []
size = 20 #number of rows in train ie. no. of images
img_size = 224 #size of the images
kernel = np.ones((2,2), np.uint8)

In [ ]:
for i in tqdm(range(size)):
    #img = load_img(os.path.join(train_dir,i), grayscale=False, color_mode='rgb', target_size=(224,224))
    img = cv2.imread(train_dir +'/' + train_csv['id_code'][i] +'.png')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_size, img_size))
    img = cv2.addWeighted (img, -5, cv2.GaussianBlur(img, (0,0), img_size/10), 5, 128)
    img = img + cv2.addWeighted (img, 5, cv2.GaussianBlur(img, (0,0), img_size/10), -5, 128)
    img = rgb2red(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    #hsv_color1 = np.asarray([0, 0, 0])
    #hsv_color2 = np.asarray([2, 2, 2])
    #img = cv2.inRange(img, hsv_color1, hsv_color2)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img = cv2.dilate(img, kernel, iterations=1)
    #img = cv2.erode(img, kernel, iterations=1)
    #img = cv2.bitwise_not(img)
    img = img_to_array(img)
    #img = img/255.0
    train+=[img]
    #trainlabel+=[train_csv[train_csv['id_code']==i[0:-4]]['diagnosis'].iat[0]]
#X = np.array(train_image)

In [ ]:
for im in tqdm(os.listdir(train_dir)):
    trainlabel+=[train_csv[train_csv['id_code']==im[0:-4]]['diagnosis'].iat[0]]

In [ ]:
train=np.array(train)
trainlabel=np.array(trainlabel)

In [ ]:
plt.imshow(train[1])

In [ ]:
print(train.shape)
print(trainlabel.shape)

In [ ]:
trainlabel2=to_categorical(trainlabel)
print(trainlabel.shape)

In [ ]:
trainx,testx,trainy,testy=train_test_split(train,trainlabel,test_size=0.2,random_state=100)

In [ ]:
#train=[]
#trainlabel=[]
#for im in tqdm(os.listdir(train_dir)):
 #   image=load_img(os.path.join(train_dir,im), grayscale=False, color_mode='rgb', target_size=(224,224))
  #  image=img_to_array(image)
   # image=image/255.0
    #train+=[image]
    #trainlabel+=[train_csv[train_csv['id_code']==im[0:-4]]['diagnosis'].iat[0]]

In [ ]:
#test=[]
#for im in tqdm(os.listdir(test_dir)):
 #   image=load_img(os.path.join(test_dir,im), grayscale=False, color_mode='rgb', target_size=(224,224))
  #  image=img_to_array(image)
   # image=image/255.0
    #test+=[image]

In [ ]:
#train=np.array(train)
#trainlabel=np.array(trainlabel)
#test=np.array(test)

#trainlabel2=to_categorical(trainlabel)

#trainx,testx,trainy,testy=train_test_split(train,trainlabel2,test_size=0.2,random_state=100)

In [ ]:
print(trainx.shape)
print(trainy.shape)
print(testx.shape)
print(testy.shape)

In [ ]:
BATCH_SIZE = 16

#def create_datagen():
 #   return ImageDataGenerator(
  #      zoom_range=0.1,  
   #     rotation_range = 30,
    #    horizontal_flip=True, 
     #   vertical_flip=True,  
    #)

# Using generator
#data_generator = create_datagen().flow(trainx, trainy, batch_size=BATCH_SIZE, seed=2019)

datagen = ImageDataGenerator(zoom_range=0.1,  
        rotation_range = 30,
        horizontal_flip=True, 
        vertical_flip=True,
        )

In [ ]:
m = applications.MobileNet(include_top=False, weights='imagenet', input_shape=(224,224,3))

In [ ]:
from keras.applications import VGG16
from keras.optimizers import Adam
from keras import layers


def build_model():
    model = Sequential()
    model.add(m)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(256))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(256))
    model.add(layers.Dense(1))
    
    model.compile(
        loss='mse',
        optimizer=Adam(lr=0.0001),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
#history = model.fit(
 #   data_generator,
  #  steps_per_epoch=trainx.shape[0] / BATCH_SIZE,
   # epochs=10,
   # validation_data = (testx, testy),
   # )

    
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from PIL import Image

import tensorflow as tf
from keras import layers
from tensorflow.keras import applications 
from keras.applications import MobileNet
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras import models


from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix

from tqdm import tqdm


#history = model.fit(
 #   data_generator,
  #  validation_data=(testx,testy),
   # epochs=5)

his=model.fit(datagen.flow(trainx,trainy,batch_size=16),validation_data=(testx,testy),epochs=20,steps_per_epoch=trainx.shape[0] / BATCH_SIZE )

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
#Make confusion matrix
#Make classification report
#Make roc curve

In [ ]:
from sklearn.metrics import confusion_matrix, cohen_kappa_score,accuracy_score
import seaborn as sns
def plot_conf_matrix(true,pred,classes):
    cf = confusion_matrix(true, pred)
    
    df_cm = pd.DataFrame(cf, range(len(classes)), range(len(classes)))
    plt.figure(figsize=(8,5.5))
    sns.set(font_scale=1.4)
    sns.heatmap(df_cm, annot=True, annot_kws={"size": 16},xticklabels = classes ,yticklabels = classes,fmt='g')
    #sns.heatmap(df_cm, annot=True, annot_kws={"size": 16})
    plt.show()
    
